In [193]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import pearsonr


%matplotlib inline

plt.rc('axes', labelsize=20) 
plt.rcParams["figure.figsize"] = (10,10)
plt.rc('xtick', labelsize=13)
plt.rc('ytick', labelsize=13) 
mpl.rcParams['axes.titlesize'] = 24

In [194]:
local_election_results_2013 = pd.read_csv('data/processed_data/local_elections_2013_results.csv')
local_election_candidates_2013 = pd.read_csv('data/processed_data/local_elections_candidates_2013.csv')

In [195]:
merged_df = pd.merge(
    local_election_results_2013, local_election_candidates_2013, 
    left_on=['INE_ID', 'Sigla', 'year'], right_on=['INE_ID', 'party', 'year'],
    how='outer')
merged_df = merged_df.drop('party', 1)

In [196]:
not_merged = merged_df[merged_df.Concelho.isnull()]

In [197]:
from numpy import math

def fill_names(line):
    if not isinstance(line['candidate_name'], basestring):
        correct_name = not_merged.ix[not_merged.INE_ID == line.INE_ID]['candidate_name']
        if len(correct_name) == 1:
            merged_df.set_value(col='candidate_name', index=line.name, value=correct_name)
        else:
            if len(correct_name) > 1:
                print 'More than one citizens party in INE_ID ' + str(line.INE_ID)
            if len(correct_name) == 0:
                print 'Zero with citizens party in INE_ID ' + str(line.INE_ID)

In [198]:
# Some hacking to merge the rest of the entries
for i in range(len(merged_df)):
    line = merged_df.ix[i]
    fill_names(line)

In [199]:
merged_df = merged_df.ix[merged_df.Concelho.notnull()]